In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from sqlite3 import connect
import pandas as pd

In [2]:
# Гуглим список основных витаминов
list_of_vitamins = [
'Biotin (vitamin B7)',
'Folic acid (folate, vitamin B9)',
'Niacin (vitamin B3)',
'Vitamin B5',
'Riboflavin (vitamin B2)',
'Thiamin (vitamin B1)',
'Vitamin B6',
'Vitamin B12',
'Vitamin C',
'Vitamin A',
'Vitamin D',
'Vitamin E',
'Calcium',
'Magnesium',
'Phosphorus',
'Potassium',
'Copper',
'Iodine',
'Iron',
'Manganese',
'Molybdenum',
'Selenium']

In [3]:
conn = connect("minerals.db")
c = conn.cursor()

c.execute("""
CREATE TABLE min_info (mineral, cause1, cause2, how, get1, get2, get3)
""")

In [4]:
# Тут мы используем selenium, чтобы получить ответы на вопросы 
# "Зачем мне нужен витамин" – why1 и why2
# (но иногда написана только одна причина, поэтому нам нужна парочка if-ов)
# "Где мы можем найти витамин?" - where (в общих словах) и
# food1, food2, food3 - три примера еды
# Из-за того, что на сайте нет стандартного вида страницы, а у параграфов нет
# особого класса, нам приходится использовать if-ы
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://ods.od.nih.gov')
for vitamin in list_of_vitamins:
    conn = connect("minerals.db")
    c = conn.cursor()
    search_button = driver.find_element_by_class_name("txt_search")
    search_button.click()
    search_button.send_keys(vitamin)
    google_search = driver.find_element_by_class_name("btn_search")
    google_search.click()
    links = driver.find_elements_by_class_name('recommend_title')
#     Нам нужна именно ссылка, которая содержит 'Consumer Fact Sheet', потому что
#     они имеют примерно одинаковое строение
    for i in links:
        if 'Consumer Fact Sheet' in i.text:
            link_to_mineral = i.find_element_by_css_selector('a').get_attribute('href') 
    driver.find_element_by_xpath(f"//a[@href='{link_to_mineral}']").click()
    pars = driver.find_elements_by_css_selector('p')
    points = driver.find_elements_by_css_selector('li')
    if 'The amount' in pars[5].text:
        why1 = pars[3].text
        why2 = pars[4].text
        food1 = points[72].text
        food2 = points[73].text
        food3 = points[74].text
        where = pars[6].text
    elif 'How much' in pars[5].text:
        why1 = pars[3].text
        why2 = pars[4].text
        food1 = points[72].text
        food2 = points[73].text
        food3 = points[74].text
        where = pars[6].text
    else:
        why1 = pars[4].text
        why2 = pars[5].text
        food1 = points[72].text
        food2 = points[73].text
        food3 = points[74].text
        where = pars[7].text
    
    if vitamin == 'Vitamin D':
        food1 = points[72].text
        food2 = points[73].text
        food3 = points[74].text
        
    if vitamin == 'Riboflavin (vitamin B2)':
        why1=''
        
    if vitamin == 'Vitamin B5':
        where = pars[7].text
    
    if vitamin == 'Vitamin C':
        where = pars[8].text
        
    if vitamin == 'Niacin (vitamin B3)':
        where = pars[7].text
        
    if vitamin == 'Folic acid (folate, vitamin B9)':
        where = pars[8].text

    info_list=[vitamin, why1, why2, where, food1,food2,food3]
    c.execute('''
    INSERT INTO min_info (mineral, cause1, cause2, how, get1, get2, get3) VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', tuple(info_list))
    conn.commit()

[WDM] - Current google-chrome version 83.0.4103
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_mac64.zip


[WDM] - Unpack archive /Users/user/.wdm/drivers/chromedriver/83.0.4103.39/mac64/chromedriver.zip
